We provide four synthetic datasets of increasing number of anomaly subgraphs. Specifically, there are  5, 30, 50 and 100 handcrafted subgraphs in the biggest connected component of transfer graph which contains more than 400,000 nodes. Each sub-graph is a connected community with random number of nodes from 20 to 200. To reduce the influence to algorithms, these subgraphs are controlled to independent and no overlap. Note that risky flows are generated within subgraphs, and the number is controlled on the 1/10 of the nodes of sub-graph. For example, generated sub-graph with 100 nodes have 10 flows. The code of generating synthetic dataset is presented at file gen_synthetic.py. A brief description of this synthetic datasets:

| Datasets   | S5      | S30     | S50     | S100    |
|------------|---------|---------|---------|---------|
| #subgraphs | 5       | 30      | 50      | 100     |
| #paths     | 33      | 272     | 429     | 651     |
| #nodes     | 477010  | 477010  | 477010  | 477010  |
| #edges     | 1066772 | 1067818 | 1068448 | 1069369 |

In [1]:
import msfrm
import json
import pandas as pd

In [2]:
def r_data(name):
    with open(f'../dataset_syn/{name}_data.json', 'r') as f:
        data = json.load(f)
        label_subgraphs = data['subgraphs']
        label_paths = data['paths']
        label_paths_ = []
        for subgraph in label_paths:
            sub_ = []
            for path in subgraph:
                sub_ += path
            label_paths_.append(sub_)

        n_subgraphs = data['n_subgraphs']
        n_paths = data['n_paths']
        concat_data = data['concat_data']
        source_data = pd.DataFrame(concat_data, columns=['from', 'to', 'date', 'amount'])
        source_data = source_data.iloc[:,[0,1,3]]

        return [label_subgraphs, label_paths_, n_subgraphs, n_paths, source_data]

In [3]:
def hit_subgraphs(label_graph, pred_graph):
    k = set()
    for i in range(len(pred_graph)):
        for j in range(len(label_graph)):
            if set(pred_graph[i]) & set(label_graph[j]):
                k.add(j)
    return k

Due to the structure difference between sub-graph and flow, we use hit accuracy as evaluation criteria of detection performance and generate amount-abnormal transactions as target risky transactions in synthetic datasets. Assumed that there are $n$ anomaly subgraphs in a transaction network, $m$ abnormal flows in these $n$ subgraphs, $n_{hit}$ describes how many anomaly subgraphs are hit in the top $n$ dense subgraphs outputted by subgraph-based algorithm or in the top $m$ dense flows outputted by our algorithm. We use $N_{pred}$ to represent the node set of algorithm outputs and $N_{truth}^i$ to represent the node set of the $i$th known anomaly subgraph. Finally, hit accuracy is defined as:
\begin{align}
    \text{accuracy} &= \frac{n_{hit}}{n} \\
    n_{hit} = \sum_{i=1}^{n} & \mathbf{I}(N_{truth}^{i} \cap N_{pred})
\end{align}
where $\mathbf{I}$ is indicator function. $\mathbf{I}=1$ when $N_{truth}^{i} \cap N_{pred} \neq \emptyset$, otherwise $\mathbf{I}=0$. Intuitively, hit accuracy is measured by the ratio between the number of hit subgraphs and total anomaly subgraphs.

In [4]:
synthetic_data = ['random5', 'random30', 'random50', 'random100']

for name in synthetic_data:
    data = r_data(name)
    k = data[3]
    output = msfrm.msfrm(data[4], k)
    hit_graphs = hit_subgraphs(data[0], [i[1] for i in output])
    n_hit = len(hit_graphs)
    accuracy = n_hit / data[2]
    print(f'Hit accuracy of {name} is {accuracy}!')

Weight vector initializing successful! Max score is 519035096.021601, subgraph is [411696, 71911].
!!!!!!!!!!!!!!

Max score update: 830452836.522721! Flow is [411696, 71911, 233915].
